In [1]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Subset
import numpy as np
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

In [2]:
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split

# CIFAR-10 데이터 불러오기
(X_train_full, y_train_full), (X_test_full, y_test_full) = cifar10.load_data()

# 데이터 정규화 (0-255 값을 0-1 사이로)
X_train_full = X_train_full.astype('float32') / 255.0
X_test_full = X_test_full.astype('float32') / 255.0

# RGB -> Grayscale 변환
# 공식: 0.299*R + 0.587*G + 0.114*B
X_train_gray = np.dot(X_train_full[...,:3], [0.299, 0.587, 0.114])
X_test_gray = np.dot(X_test_full[...,:3], [0.299, 0.587, 0.114])

# 원하는 두 개의 클래스만 선택 (예: 클래스 0과 1)
selected_classes = [0, 1]

# 클래스 0과 1에 해당하는 데이터만 선택 (train set)
train_mask = np.isin(y_train_full, selected_classes)
X_train_filtered = X_train_gray[train_mask.squeeze()]
y_train_filtered = y_train_full[train_mask.squeeze()]

# 클래스 0과 1에 해당하는 데이터만 선택 (test set)
test_mask = np.isin(y_test_full, selected_classes)
X_test_filtered = X_test_gray[test_mask.squeeze()]
y_test_filtered = y_test_full[test_mask.squeeze()]

# 클래스 라벨을 이진 라벨로 변환 (0 또는 1로)
y_train_filtered = (y_train_filtered == selected_classes[1]).astype(int)
y_test_filtered = (y_test_filtered == selected_classes[1]).astype(int)

# 시드 고정
np.random.seed(42)

# 2000개의 데이터를 무작위로 선택
num_samples = 200
indices = np.random.choice(len(X_train_filtered), num_samples, replace=False)
X_sampled, y_sampled = X_train_filtered[indices], y_train_filtered[indices]

# 2000개의 샘플에서 train/test 데이터 분할 (80% train, 20% test 비율로 나눔)
X_train, X_test, y_train, y_test = train_test_split(
    X_sampled, y_sampled, stratify=y_sampled, test_size=0.2, random_state=42
)

y_train = y_train.squeeze(1)
y_test = y_test.squeeze(1)

# 결과 출력
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print(f"Sampled train labels: {np.unique(y_train)}")
print(f"Sampled test labels: {np.unique(y_test)}")

2024-10-24 22:19:07.558692: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 22:19:07.560857: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-24 22:19:07.707203: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-24 22:19:07.956652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 22:19:08.426762: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

Training set size: 160
Test set size: 40
Sampled train labels: [0 1]
Sampled test labels: [0 1]


In [3]:
# 2x2 크기로 겹치지 않게 패치로 나누는 함수
def split_into_non_overlapping_patches(image, patch_size=(4, 4)):
    patches = []
    for i in range(0, image.shape[0], patch_size[0]):
        for j in range(0, image.shape[1], patch_size[1]):
            patch = image[i:i+patch_size[0], j:j+patch_size[1]].flatten()
            patches.append(patch)
    return np.array(patches)

In [4]:
# 각 이미지를 2x2 겹치지 않는 패치로 나누기
X_train = np.array([split_into_non_overlapping_patches(img) for img in X_train])
X_test = np.array([split_into_non_overlapping_patches(img) for img in X_test])

In [5]:
# y_train이 numpy 배열일 경우
unique, counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

{0: 80, 1: 80}


In [6]:
# y_train이 numpy 배열일 경우
unique, counts = np.unique(y_test, return_counts=True)
class_distribution = dict(zip(unique, counts))
print(class_distribution)

{0: 20, 1: 20}


In [7]:
X_train.shape

(160, 64, 16)

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Self-Attention and Binary Classifier from previous code
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert self.head_dim * heads == embed_size, "Embedding size must be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask=None):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split embedding into multiple heads
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

# Classifier model
class SimpleSelfAttentionClassifier(nn.Module):
    def __init__(self, embed_size=16, heads=2, num_classes=2):
        super(SimpleSelfAttentionClassifier, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.fc = nn.Linear(embed_size * 64, num_classes)  # 49 is the sequence length in X_train

    def forward(self, x):
        attention_output = self.attention(x, x, x)
        attention_output = attention_output.flatten(start_dim=1)  # Flatten to feed into classifier
        out = self.fc(attention_output)
        return out

In [14]:
# Model, loss, optimizer
model = SimpleSelfAttentionClassifier(embed_size=16, heads=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
# def binary_accuracy(preds, y):
#     """
#     Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
#     """

#     #round predictions to the closest integer
#     rounded_preds = (torch.round(torch.sign(preds-0.5))+1)//2
#     correct = (rounded_preds == y).float() #convert into float for division 
#     acc = correct.sum() / len(correct)
#     return acc

In [16]:
# Define your binary accuracy function
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    
    return acc

In [17]:
# Training the model
for iepoch in tqdm(range(30)):
    optimizer.zero_grad()
    
    # Convert your X_train into tensor
    X_tensor = torch.tensor(X_train, dtype=torch.float32)
    
    # Forward pass
    predictions = model(X_tensor)
    
    # Convert y_train to tensor and change labels from 1 to 0, and 7 to 1
    label = torch.tensor(y_train, dtype=torch.long).clone()  # Ensure labels are LongTensor
    for i in range(len(label)):
        label[i] = 0 if label[i] == 1 else 1  # 1 -> 0, 7 -> 1
    
    # Compute loss
    loss = criterion(predictions, label)
    
    # Calculate binary accuracy (adjusted for your binary classification)
    acc = binary_accuracy(predictions.argmax(dim=1), label)
    
    # Print accuracy and loss
    print(f'Accuracy: {acc}')
    print(f'Loss: {loss.item()}')
    
    # Backward pass and optimization step
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {iepoch+1} complete")


  0%|          | 0/30 [00:00<?, ?it/s]

Accuracy: 0.5
Loss: 0.6940199136734009


  3%|▎         | 1/30 [00:08<04:14,  8.78s/it]

Epoch 1 complete
Accuracy: 0.5
Loss: 0.7460027933120728


  7%|▋         | 2/30 [00:17<04:09,  8.92s/it]

...

Epoch 28 complete
Accuracy: 0.612500011920929
Loss: 0.666343629360199


 97%|█████████▋| 29/30 [02:38<00:04,  4.59s/it]

Epoch 29 complete
Accuracy: 0.6312500238418579
Loss: 0.6660415530204773


100%|██████████| 30/30 [02:41<00:00,  5.38s/it]

Epoch 30 complete


In [20]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, average_precision_score

# Convert test data to tensor
X_tensor = torch.tensor(X_test, dtype=torch.float32)
predictions = model(X_tensor)

# Convert y_train to tensor and change labels from 1 to 0, and 7 to 1
label = torch.tensor(y_test, dtype=torch.long).clone()  # Ensure labels are LongTensor
        
# Reverse the labels (if needed)
for i in range(len(label)):
    if label[i] == 1:
        label[i] = 0
    else:
        label[i] = 1

# Calculate loss and accuracy
# Compute loss
loss = criterion(predictions, label)

# Calculate binary accuracy (adjusted for your binary classification)
acc = binary_accuracy(predictions.argmax(dim=1), label)

# Convert predictions to probabilities using softmax
probabilities = torch.softmax(predictions, dim=1)  # Shape: (batch_size, 2)

# Extract probabilities for class 1 (positive class)
positive_class_probs = probabilities[:, 1].detach().numpy()

# Convert labels to numpy
labels_np = label.numpy()

# Calculate AUROC
auroc = roc_auc_score(labels_np, positive_class_probs)

# Binarize predictions for precision, recall, and F1 calculation
binary_preds = np.where(positive_class_probs > 0.5, 1, 0)

# Calculate Precision, Recall, and F1 Score
precision, recall, f1, _ = precision_recall_fscore_support(labels_np, binary_preds, average='binary')

# Calculate AUPRC (Area Under the Precision-Recall Curve)
auprc = average_precision_score(labels_np, positive_class_probs)

# Print results
print(f'\nAccuracy: {acc}\n')
print(f'Loss: {loss}\n')
print(f'AUROC: {auroc}\n')
print(f'Precision: {precision}\n')
print(f'Recall: {recall}\n')
print(f'F1 Score: {f1}\n')
print(f'AUPRC: {auprc}\n')


Accuracy: 0.625

Loss: 0.6493237614631653

AUROC: 0.685

Precision: 0.631578947368421

Recall: 0.6

F1 Score: 0.6153846153846154

AUPRC: 0.6729369268219667

